In [9]:
from transformers import AutoModelForCausalLM, AutoModelForSeq2SeqLM, AutoTokenizer

In [10]:
summarize_model_name: str = "digit82/kobart-summarization"
summarize_num_beams: int = 4
sum_input_max_length: int = 512
sum_max_token_length: int = 128

model_name: str = "../models/polyglot-1.3b-11250steps-with-sum"
tokenizer_name: str = model_name
prompt_max_length = 128
max_length: int = 512
device: str = "cuda"
auth_token: str = None

num_generate_tokens: int = 5000

In [11]:
tokenizer_kwargs = {"padding_side": "left", "truncation_side": "left"}
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, use_auth_token=auth_token, **tokenizer_kwargs)
model = AutoModelForCausalLM.from_pretrained(model_name, use_auth_token=auth_token).to(device)

In [12]:
tokenizer_kwargs = {"padding_side": "left", "truncation_side": "left"}
summarize_tokenizer = AutoTokenizer.from_pretrained(summarize_model_name, use_auth_token=auth_token, **tokenizer_kwargs)
summarize_model = AutoModelForSeq2SeqLM.from_pretrained(summarize_model_name).to(device)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [13]:
def datum_to_string(datum, use_n_summary: int = 10):
    text = f"제목: {datum['title']}\n"

    summarizations = datum.get("summarizations")
    if summarizations:
        summarizations = summarizations[-use_n_summary:]
        summarization = " ".join(summarizations).replace("\n", " ")
        text = f"요약: {summarization}\n" + text
    return text


input_examples = [
    {"title": "나의 빛과 어둠", "summarizations": [], "content": ""},
]

In [14]:
total_tokens = 0

while total_tokens < num_generate_tokens:
  prompts = [datum_to_string(example) for example in input_examples]
  input_ids = tokenizer(
          prompts,
          add_special_tokens=False,
          max_length=prompt_max_length,
          padding="longest",
          truncation=True,
          return_tensors="pt",
          return_token_type_ids=False,
          return_attention_mask=False,
  )["input_ids"].to(device)

  outputs = model.generate(input_ids, 
      max_length, 
      do_sample=True,
      num_return_sequences=1,
      # repetition_penalty=1.2,
      # num_beams=5,
      # temperature=2.0,
      # top_p=0.,
      # top_k=3,
      pad_token_id=tokenizer.pad_token_id,
      use_cache=True,
  )
  outputs = outputs[:, input_ids.size(1):]
  total_tokens += outputs.size(1)

  texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
  # Remove last incomplete sentence
  texts = [text[:max(text.rfind(last_char) for last_char in ".?!\n")] for text in texts]
  for example, text in zip(input_examples, texts):
    example["content"] += text

  sum_input_ids = summarize_tokenizer(
          texts,
          add_special_tokens=False,
          max_length=sum_input_max_length,
          padding="longest",
          truncation=True,
          return_tensors="pt",
          return_token_type_ids=False,
          return_attention_mask=False,
  )["input_ids"].to(device)

  sum_outputs = summarize_model.generate(sum_input_ids, 
      sum_max_token_length, 
      num_beams=summarize_num_beams,
      num_return_sequences=1,
      eos_token_id=summarize_tokenizer.eos_token_id,
      pad_token_id=summarize_tokenizer.pad_token_id,
      use_cache=True,
  )

  summarizations = summarize_tokenizer.batch_decode(sum_outputs, skip_special_tokens=True)

  for example, summarization in zip(input_examples, summarizations):
    example["summarizations"].append(summarization)

In [15]:
for example in input_examples:
    print(f'제목: {example["title"]}\n\n')
    print(example["content"])

제목: 나의 빛과 어둠







하루에도 몇 번씩 나는 나를 찾아서 나에게 물었다. 

"너의 오늘은 어때?"
"너를 위해 한 일이 좋아?"
"너는 어떤 사람인지 궁금해"
"너는 언제 행복할 것 같아?"
"너는 지금 뭐가 기분이 좋은지 말해줄래?"














﻿'나는 어떤 인물인가'에 대해서 말해보려고 한다.

나는 생각보다 내가 하고 싶은 것, 잘하는 것을 확실히 아는 편이다. 그것과 더불어 내가 좋아하는지 싫어하는지에 관해서도 뚜렷하게 구분한다. 사람들에게 '내가 좋아하는 것과 싫어하는 것'을 얘기하라고 하면 반반 정도인데, 반반인 것이 오히려 더 복잡해서, 가끔은 너무 분명해서 오히려 모호하게 느껴질 때도 있다. 내가 좋아하는 것과 싫어하는 것에 관해서는 나름의 일관성이 있는데, 그것에 대해서는 명확하게 얘기하기가 어려워서 이런 경우는 그냥 남들한테 "너는 이런 걸 하고 싶다."라는 식으로 얘기한다. 

어떤 면에서는 "예쁜" 사람들이고, 또 어떤 면에서는 그렇지 않은 데도 불구하고, 나는 이런 사람에게는 "왜 그렇게 예쁘게 생겼어요?"와 같은 말은 한 번도 한 적 없으며, 늘 '사람들이 그렇게 보지만 내 기준으론 그냥 자연스럽고, 예뻐.'라고 말한다. 

그러나 모든 사람들에게 그렇게 하는 것은, 마치 내 머릿속에 '예쁘다'라는 말이 있고 그 말과 생각으로 나는 예쁜가? 그렇지 않은지도 생각해 보자는 의미도 있다. 
【서울=뉴시스】 서울에 사는 동안 단 한 번도 살던 곳을 떠난 적이 없었다. 집은 그대로, 사람의 삶도 그대로였다. 하지만 조금씩 변해가는 세상만큼 나도 변화했을 테니 떠나야만 하는 사람이었지만 떠나지 못하고 집에 남아도는 것들이 그렇게도 아쉬웠을까.
So happy to finally finish그러니 좋아하는 것과 싫어하는 것을 분리해서 생각하는 것은 어쩌면 자연스러운 일이었을지도 모르겠다.
어릴 때는 엄마를 보며 좋아하고,
요즘에는 엄마를 보면 왜 이렇게 안 좋은지 모르겠지만 적어도 내가 어릴 때보다는 많다고 해야 할

In [16]:
for example in input_examples:
    print("\n".join(example["summarizations"]))
    print()

'나는 좋아하는 것과 싫어하는 것'을 얘기하라고 하면 반반 정도인데, 반반인 것이 오히려 더 복잡해서 오히려 모호하게 느껴질 때도 있다.
좋아하는 것과 싫어하는 것을 분리해서 생각하는 것은 어쩌면 자연스러운 일이었을지도 모르겠다.
좋아하는 것만큼 싫어하는 것도 명확해야 하며 좋아하는 것만큼 싫어하는 것도 명확해야 한다.
지금은 ‘나’도 아니고 ‘너’도 아닌, 애매한 상태를 즐긴다.
'내가 사랑한다고 해서 상대방이 상처를 받고, '네가 사랑한다고 해서 네가 아파할' 것이라는 생각과 '내가 사랑한다고 해서 상대방이 나를 사랑하지 않고, '네가 사랑한다고 해서 네가 상처를 받아야' 할 것이라는 생각 등을 머릿속에 끊임없이 그리며 살던 지난날들이 너무나 어두웠다는 사실에 부끄러움을 느꼈다.
사람마다 각자의 방식대로 ‘내가 사랑하는 방식’과 ‘네가 나를 사랑하는 방식’, 그리고 ‘네가 이해하지 못하는 너의 행동이’ 있다는 의미인 동시에 ‘타인이 나를 사랑하는 방식’과 ‘네가 나를 사랑하는 방식’, 그리고 ‘네가 이해하지 못하는 너의 행동이’ 있다는 의미인 동시에 ‘타인이 나를 사랑하는 방식’과 ‘네가 나를 사랑하는 방식’, 그리고 ‘네가 이해하지 못하는 너의 행동이’ 있다는 의미인 동시에 ‘타인이 나를 사랑하는 방식’과 ‘네가 나를 사랑하는 방식’, 그리고 ‘네가 이해하지 못하는 너의 행동이’ 있다는 의미인 동시에 ‘타인이 나를 사랑하는 방식’과
타인의 어두움 속에서 함께 걸으며 나의 빛이 타인의 어둠 속에서도 함께하길 바라는 마음에서 나의 사랑이 누군가의 어둠 속에서도 같은 빛을 낼 수 있게 된다는 것을 증명해줄 수 있는 관계라는 점과 나의 빛이 누군가의 어둠 속에서도 함께하길 바라는 마음에서 나의 사랑이 누군가의 어둠 속에서도 같은 빛을 낼 수 있게 된다는 것을 증명해줄 수 있는 관계라는 점을 증명해줄 수 있는 관계라는 점을 확인하고 싶다.
본 고안에 따른 자동변속기용 캐리어는 상기 캐리어의 주행을 정지시키고, 상기 정지를 감속한 상태에서는 캐리어의 주행을 정지스위